In [135]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import seaborn as sns
import numpy as np
import tensorflow as tf
import pandas as pd
import os
from tensorflow import keras

In [141]:
def print_logits (y,batch_size,nb_label):
    # Print a the label predicted
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    for i in range (0,batch_size):
        print("------------",i,"------------")
        for j in range (0,nb_label):
            if (y[i][j] != 0):
                print("prédiction : ",j,"pourcentage de certitude :","%.1f" % (y[i][j]*100) , " %")

def int_to_dummies(i):
    # Create a label for the number given
    #Input: int The value of the label
    result = [0]*3
    result[int(i)] = 1
    return np.array(result)

def create_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
  ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[
                      keras.metrics.TruePositives(name='tp'),
                      keras.metrics.FalsePositives(name='fp'),
                      keras.metrics.TrueNegatives(name='tn'),
                      keras.metrics.FalseNegatives(name='fn'), 
                      keras.metrics.BinaryAccuracy(name='accuracy'),
                      keras.metrics.Precision(name='precision'),
                      keras.metrics.Recall(name='recall'),
                      keras.metrics.AUC(name='auc')])

    return model



def prediction_logits (y,batch_size,nb_label):
    # Normalize the batch of label to obtain a percentage
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    #Out : Numpy array [batch_size][10] the input normalized
    
    y_sum = np.sum(y, axis=1)
    for i in range (0,batch_size):
        for j in range (0,nb_label):
            if (y_sum[i] != 0):
                y[i][j] = y[i][j]/y_sum[i]
    
    return y


In [143]:
df = pd.read_csv("parsed_train_data.csv")
df = df[df.ID_FTR != -1]

target = df["ID_FTR"].apply(int_to_dummies)
features = df[df.columns[7:]]

In [162]:
list(y_train)
a = np.array([])
for i in list(y_train):
    a = np.append(a,i)
a = a.reshape(-1,3)

In [163]:
a

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [145]:
x_test = features.iloc[3550:].values
y_test = target.iloc[3550:].values

x_train = features.iloc[:3550].values
y_train = target.iloc[:3550].values

In [146]:
x_train = x_train.astype(float)

In [ ]:
model = create_model()

checkpoint_path = "models/model1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


model.fit(x_train,a, epochs=20,validation_data=(x_test,y_test),
          callbacks=[cp_callback],
         class_weight = {0: 1.5, 1 : 1.3, 2 : 1 })

In [132]:
index = np.random.randint(0,107)
print(y_test[index])
print_logits(model.predict(x = x_test[index].reshape(1,14), batch_size = 1),1,3)

1
------------ 0 ------------
prédiction :  0 pourcentage de certitude : 42.6  %
prédiction :  1 pourcentage de certitude : 25.0  %
prédiction :  2 pourcentage de certitude : 32.4  %


In [24]:
y_pred = model.predict(x = x_train)
y_pred = np.argmax(y_pred, axis=1)
conf_mat = tf.math.confusion_matrix(y_train, y_pred)